<a href="https://colab.research.google.com/github/yoonyerin/find_my_lover/blob/master/%EB%8F%99%EB%AC%BC%EC%83%81%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <애인상 테스트를 위한 동물상 테스트>
## 1. 실제 동물 classification에 사람 얼굴을 넣어 그 동물을 닮은 사람을 찾는 net을 구성하고자 함 
###=> 걱정되는 부분: 그냥 인간과 종간 유사도가 비슷한 동물들만 치중되어 등장할 가능성이 있다. => 이걸 하나의 target으로만 예측되는 것을 막도록 cost function을 제작하거나 score값에 변형을 주는 방식으로 접근할 수도.
## => 걱정했지만, 생각보다 문제없이 학습되는 것을 확인함. 배경의 영향을 안받도록 배경을 투명처리 또는 화이트 처리한 걸로 학습을 시키면 더 좋지 않을까 기대 중. 
## 2. 또는, 그냥 "강아지상 연예인" 이런식으로 스크레이핑 한 것들을 데이터 셋으로 사용하여 라벨을 가질 수도 있다.=>사람들의 보편적인 시각을 학습할 가능성이 높다. 

## 3. 둘 다 만들어보는 것도 재밌겠다.

# <데이터>
## 1. Kaggle animal data set
## 2. target:
badger
bat
bear
bee
beetle
bison
boar
butterfly
cat
caterpillar
chimpanzee
cockroach
cow
coyote
crab
crow
deer
dog
dolphin
donkey
dragonfly
duck
eagle
elephant
flamingo
fly
fox
goat
goldfish
goose
gorilla
grasshopper
hamster
hare
hedgehog
hippopotamus
hornbill
horse
hummingbird
hyena
jellyfish
kangaroo
koala
ladybugs
leopard
lion
lizard
lobster
mosquito
moth
mouse
octopus
okapi
orangutan
otter
owl
ox
oyster
panda
parrot
pelecaniformes
penguin
pig
pigeon
porcupine
possum
raccoon
rat
reindeer
rhinoceros
sandpiper
seahorse
seal
shark
sheep
snake
sparrow
squid
squirrel
starfish
swan
tiger
turkey
turtle
whale
wolf
wombat
woodpecker
zebra

# 우리가 사용하기로 결정한 동물상
## bear, cat, dog, deer, donkey, koala, lion,  wolf, cow, fox, goat, horse, kangaroo, mouse, owl, shark, sheep, snake, turtle

In [ ]:
pip install wandb

In [ ]:
import json
import os

In [ ]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cpu'
PROJECT_NAME = 'Animal-Clf'

In [ ]:
#pip install kaggle --upgrade

In [ ]:
#api_key_path="/content/drive/MyDrive/kaggle.json"

In [ ]:
'''with open(api_key_path) as f:
  api=json.load(f)
  os.environ["KAGGLE_USERNAME"]=api["username"]
  os.environ["KAGGLE_KEY"]=api["key"]'''

In [ ]:
#!kaggle datasets download -d iamsouravbanerjee/animal-image-dataset-90-different-animals

In [ ]:
#!kaggle kernels output programmerrdai/animal-clf -p /path/to/dest

In [ ]:
labels=["bear", "cat", "dog", "deer", "donkey", "koala", "lion",  "wolf", "cow", "fox", "goat", "horse", "kangaroo", "mouse", "owl", "shark", "sheep", "snake", "turtle"]

# 1. Data Load

In [ ]:
def load_data():
  
    labels_l = {}
    labels_r = {}
    idx = 0
    data = []
    for folder in os.listdir('/content/drive/MyDrive/Find my lover/animal/animals/animals'):
      if folder in labels:
        idx += 1
        labels_l[folder] = idx
        labels_r[idx] = folder
    for folder in tqdm(os.listdir('/content/drive/MyDrive/Find my lover/animal/animals/animals')):
        if folder in labels:
          for file in os.listdir(f'/content/drive/MyDrive/Find my lover/animal/animals/animals/{folder}/'):
              img = cv2.imread(f'/content/drive/MyDrive/Find my lover/animal/animals/animals/{folder}/{file}')
              img = cv2.resize(img,(56,56))
              img = img / 255.0
              data.append([
                  img,
                  np.eye(
                      labels_l[folder]+1,len(labels_l)
                  )[labels_l[folder]]
              ])
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
    X_train = torch.from_numpy(np.array(X_train)).to(device).view(-1,3,56,56).float()
    y_train = torch.from_numpy(np.array(y_train)).to(device).float()
    X_test = torch.from_numpy(np.array(X_test)).to(device).view(-1,3,56,56).float()
    y_test = torch.from_numpy(np.array(y_test)).to(device).float()
    return X,y,X_train,X_test,y_train,y_test,labels_l,labels_r,idx,data

# 2. Men Data Load

In [ ]:
def load_men_data():
    data=[]    
    folder_path='/content/drive/MyDrive/Find my lover/men/'
    for i in range(50):
      img_path=str(i+1)+".jpg"
      img = cv2.imread(folder_path+img_path)
      img = cv2.resize(img,(56,56))
      img = img / 255.0
      data.append(img)
                    
    men= torch.from_numpy(np.array(data)).to(device).view(-1,3,56,56).float()
    
    return men

# 3. loss, accuracy, Returnig Aninal_type of image

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        y_batch = int(torch.argmax(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc
def get_animal_type(model, X):
  pred = model(X)
  pred = int(torch.argmax(pred))
  return pred

# 4. Model

In [ ]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,7,(5,5))
        self.conv1bn = BatchNorm2d(7)
        self.conv2 = Conv2d(7,14,(5,5))
        self.conv2bn = BatchNorm2d(14)
        self.conv3 = Conv2d(14,21,(5,5))
        self.conv3bn = BatchNorm2d(21)
        self.linear1 = Linear(21*3*3,256)
        self.linear1bn = BatchNorm1d(256)
        self.linear2 = Linear(256,512)
        self.linear2bn = BatchNorm1d(512)
        self.linear3 = Linear(512,256)
        self.linear3bn = BatchNorm1d(256)
        self.output = Linear(256,len(labels))
    
    def forward(self,X):
        preds = self.max_pool2d(self.activation(self.conv1bn(self.conv1(X))))
        preds = self.max_pool2d(self.activation(self.conv2bn(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3bn(self.conv3(preds))))
        preds = preds.view(-1,21*3*3)
        preds = self.activation(self.linear1bn(self.linear1(preds)))
        preds = self.activation(self.linear2bn(self.linear2(preds)))
        preds = self.activation(self.linear3bn(self.linear3(preds)))
        preds = self.output(preds)
        return preds

# 5. train

In [ ]:
X,y,X_train,X_test,y_train,y_test,labels_l,labels_r,idx,data = load_data()

In [ ]:
model=Model().to(device)
criterion=MSELoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
batch_size=32
epochs=250

In [ ]:

for _ in tqdm(range(epochs)):
     for i in range(0,len(X_train),batch_size):
         X_batch = X_train[i:i+batch_size]
         y_batch = y_train[i:i+batch_size]
         model.to(device)
         preds = model(X_batch)
         loss = criterion(preds,y_batch)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         torch.cuda.empty_cache()
     torch.cuda.empty_cache()
     model.train()
wandb.finish()

# 6. image load for check result

In [ ]:
img_list=[]    
folder_path='/content/drive/MyDrive/Find my lover/men/'
for i in range(50):
  img_path=str(i+1)+".jpg"
  img = cv2.imread(folder_path+img_path)
  img_list.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


# 7. Value of loss and accuracy

In [ ]:
get_loss(model,X_train,y_train,criterion)

In [ ]:
get_loss(model,X_test,y_test,criterion)

In [ ]:
get_accuracy(model,X_train, y_train)

In [ ]:
predict=model(load_men_data())

In [ ]:
fig, axs=plt.subplots(10, 5, figsize=(30, 100))
for i in range(10):
  for j in range(5):
    axs[i, j].imshow(img_list[i*5+j])
    axs[i, j].set_title(labels[torch.argmax(predict, axis=1)[i*5+j]], size=20)